# MGIMO intensive

## NTL dataset: baseline model

### 1. Library installation

Documentation for use of OpenCV with Python API [see here](https://docs.opencv.org/).

In [ ]:
!pip3 install opencv-python
!pip install -U ydata-profiling

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from ydata_profiling import ProfileReport

import re
from typing import Tuple, Dict, List, Optional, Union

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from PIL import Image, ImageFile
import gc

# Включение поддержки загрузки больших изображений
ImageFile.LOAD_TRUNCATED_IMAGES = True

### 2. Explore the data

Take the dataset from Kaggle - [Country-Wise Nightlight Images Dataset](https://www.kaggle.com/datasets/abhijeetdtu/country-nightlight-dataset).

In [ ]:
DATA_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt'

In [ ]:
!ls -la $DATA_PATH

In [ ]:
IMG_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt/dataset'
TARGET_SIZE = (128, 128)
LIMIT_IMGS = 500
RANDOM_STATE = 42
PROFILE = False

In [ ]:
data_dir = Path(IMG_PATH)

### 3. Panel data

In [ ]:
df_gdp = pd.read_csv(f"{DATA_PATH}/gdp_melted.csv", index_col=0)

In [ ]:
if PROFILE:
    profile = ProfileReport(
        df_gdp, 
        title="GDP data profiling report"
    )
    profile

In [ ]:
df_gdp.info()

In [ ]:
df_gdp.head()

### 4. Images data

In [ ]:
# 1. Count files and images
all_files = list(data_dir.glob('*'))
image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}
image_files = [f for f in all_files if f.suffix.lower() in image_extensions]

print(f"Total files: {len(all_files)}")
print(f"Image files: {len(image_files)}")

In [ ]:
# 2. Open and display a few images
if image_files:
    # Display up to 5 images
    num_to_show = min(4, len(image_files))
    
    fig, axes = plt.subplots(1, num_to_show, figsize=(15, 5))
    if num_to_show == 1:
        axes = [axes]
    
    for i, img_file in enumerate(image_files[:num_to_show]):
        # Read image with OpenCV
        img_path = str(img_file)
        img = cv2.imread(img_path)
        
        if img is not None:
            # Convert BGR to RGB for matplotlib
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Display image
            axes[i].imshow(img_rgb)
            axes[i].set_title(f"{img_file.name}\n{img.shape}")
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, f"Could not load\n{img_file.name}", 
                       ha='center', va='center')
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print image details
    print("\nFirst few images details:")
    for img_file in image_files[:num_to_show]:
        img = cv2.imread(str(img_file))
        if img is not None:
            print(f"  {img_file.name}: shape={img.shape}, size={img_file.stat().st_size / 1024:.1f} KB")

### 5. Simple regression model

Will explore how GDP can be predicted by NTL images.

Use prompt to generate code for modelling:

```prompt
## Базовые параметры
- Роль: Junior Python Developer (Data Analyst, CV)
- Специализация: data analysis, computer vision
- Уровень: начинающий
- Температура: 0 (максимальная точность и предсказуемость)

## Контекст выполнения
Разработка кода для построения модели прогноза на основе изображений

## Входные данные
- Pandas датафрейм `df_gdp` c данными 
- Изображения в формате JPEG

## Технические ограничения
- Использовать стандартные библиотеки Python
- Код запускается в интерактивном ноутбуке Jupyter
- Необходимо учесть производительность, количество изображений около 3 тысяч, размер изображений примерно 3000 на 7000 пикселей

## Требования к реализации

### Функциональные требования
1. Требуется создание датафрейма `df_img` на основе изображений c с полями, позволяющими соединить этот датафрейм с `df_gdp` датафрейме
2. Датафрейм `df_img` должен содержать поля, позволяющие соединить его с `df_gdp`, поле с путями к файлам, данные полученные из названия файла изображения (год, код страны) 
3. Изображения необходимо уменьшить в разрешении, чтобы ускорить обработку
4. Вектор, полученный из каждого изображения будет формировать список признаков, на основе которого будет предсказываться показатель GDP из датафрейма `df_gdp` 
5. При разделении на обучающую и тестовую выборку используй разделение по странам и временным периодам, для минимизации data leaks
6. Для демонстрации использую простую линейную регрессию 

### Технические требования
- Сложность кода: не используй классы, ограничься функциями
- Архитектура: упрощенная, для использования в интерактивных ноутбуках
- Стиль кода: PEP 8, black (длина строки 79)
- Документация: Docstrings в стиле Google
- Безопасность: Никаких захардкоженных credentials

### Структура данных `df_gdp`
<class 'pandas.core.frame.DataFrame'>
Index: 16104 entries, 0 to 16103
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         16104 non-null  object 
 1   code            16104 non-null  object 
 2   indicator       16104 non-null  object 
 3   indicator_code  16104 non-null  object 
 4   year            16104 non-null  int64  
 5   gdp             12372 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 880.7+ KB

### Пример данных `df_gdp`
  country code  indicator indicator_code  year  gdp
0 Aruba ABW GDP (current US$) NY.GDP.MKTP.CD  1960  NaN
1 Afghanistan AFG GDP (current US$) NY.GDP.MKTP.CD  1960  537777811.1
2 Angola  AGO GDP (current US$) NY.GDP.MKTP.CD  1960  NaN
3 Albania ALB GDP (current US$) NY.GDP.MKTP.CD  1960  NaN
4 Andorra AND GDP (current US$) NY.GDP.MKTP.CD  1960  NaN

### Хранение изображений
IMG_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt/dataset'

### Пример наименования файлов изображений
  CHN_1997.png_0_4784.jpeg: shape=(4382, 7905, 3), size=768.3 KB
  JPN_2005.png_0_1141.jpeg: shape=(2393, 3296, 3), size=152.8 KB
  AUS_2008.png_0_7320.jpeg: shape=(4153, 5543, 3), size=492.8 KB
  CHN_2001.png_0_3134.jpeg: shape=(4382, 7905, 3), size=783.3 KB
```

In [ ]:
def create_image_dataframe(img_path: str) -> pd.DataFrame:
    """Создает датафрейм с информацией об изображениях.
    
    Args:
        img_path: Путь к директории с изображениями
        
    Returns:
        DataFrame с колонками:
        - country_code: код страны из названия файла
        - year: год из названия файла
        - file_path: полный путь к файлу изображения
        - filename: исходное имя файла
        
    Raises:
        FileNotFoundError: Если указанная директория не существует
    """
    
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Директория {img_path} не найдена")
    
    # Собираем информацию о файлах
    image_data = []
    
    # Паттерн для извлечения кода страны и года из имени файла
    # Пример: CHN_1997.png_0_4784.jpeg
    pattern = re.compile(r'([A-Z]{3})_(\d{4})')
    
    for filename in os.listdir(img_path):
        if not filename.lower().endswith(('.jpeg', '.jpg', '.png')):
            continue
            
        match = pattern.search(filename)
        if match:
            country_code = match.group(1)
            year = int(match.group(2))
            
            image_data.append({
                'country_code': country_code,
                'year': year,
                'file_path': str(Path(img_path) / filename),
                'filename': filename
            })
    
    df_img = pd.DataFrame(image_data)
    
    if df_img.empty:
        print("Предупреждение: не найдено изображений, соответствующих паттерну")
    
    return df_img


def load_and_resize_image(file_path: str, target_size: Tuple[int, int] = (224, 224)) -> np.ndarray:
    """Загружает изображение и изменяет его до точного размера target_size.
    
    Args:
        file_path: Путь к файлу изображения
        target_size: Целевой размер (ширина, высота), по умолчанию (224, 224)
        
    Returns:
        Изображение в виде numpy массива (224, 224) с типом float16
    """
    try:
        # Загружаем изображение сразу в grayscale
        img = cv2.imread(file_path, cv2.IMREAD_REDUCED_GRAYSCALE_2)
        
        if img is None:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            
        if img is None:
            print(f"Не удалось загрузить изображение: {file_path}")
            return None
        
        # Принудительный ресайз до точного размера 224x224
        # INTER_LINEAR хорош для увеличения, INTER_AREA для уменьшения
        # Определяем, нужно увеличивать или уменьшать
        height, width = img.shape[:2]
        if height > target_size[1] or width > target_size[0]:
            # Уменьшаем
            interpolation = cv2.INTER_AREA
        else:
            # Увеличиваем
            interpolation = cv2.INTER_LINEAR
        
        # Ресайз до точного размера
        img_resized = cv2.resize(img, target_size, interpolation=interpolation)
        
        # Конвертируем в float16 и нормализуем
        img_array = img_resized / 255.0
        
        # Очищаем память
        del img, img_resized
        gc.collect()
        
        return img_array
        
    except Exception as e:
        print(f"Ошибка при загрузке {file_path}: {e}")
        return None


def extract_image_features(
    image_array: np.ndarray,
    method: str = 'flatten'
) -> np.ndarray:
    """Извлекает признаки из черно-белого изображения.
    
    Args:
        image_array: Изображение в виде numpy массива (height, width)
        method: Метод извлечения признаков
        
    Returns:
        Вектор признаков
    """
    if image_array is None:
        return None
    
    if method == 'flatten':
        # Для grayscale просто выпрямляем 2D массив
        features = image_array.flatten()
    elif method == 'mean_pooling':
        # Усреднение по блокам для уменьшения размерности
        h, w = image_array.shape
        block_size = 8
        h_blocks = h // block_size
        w_blocks = w // block_size
        features = image_array[:h_blocks*block_size, :w_blocks*block_size] \
                          .reshape(h_blocks, block_size, w_blocks, block_size) \
                          .mean(axis=(1, 3)) \
                          .flatten()
    else:
        raise ValueError(f"Неизвестный метод: {method}")
    
    return features


def prepare_features_for_country_year(
    df_img: pd.DataFrame,
    method: str = 'flatten',
    limit: int = 200,
) -> pd.DataFrame:
    """Подготавливает признаки для всех изображений.
    
    Args:
        df_img: Датафрейм с информацией об изображениях
        method: Метод извлечения признаков
        
    Returns:
        DataFrame с признаками и мета-информацией
    """
    
    feature_list = []

    if limit == None: limit = len(df_img)
    
    for idx, row in df_img[:limit].iterrows():
        if idx % 100 == 0:  # Прогресс каждые 100 изображений
            print(f"Обработано {idx} из {len(df_img)} изображений")
        
        # Загружаем и уменьшаем изображение
        img_array = load_and_resize_image(row['file_path'], TARGET_SIZE)
        
        if img_array is not None:
            # Извлекаем признаки
            features = extract_image_features(img_array, method)
            
            # Сохраняем признаки вместе с мета-информацией
            feature_dict = {
                'country_code': row['country_code'],
                'year': row['year'],
                'file_path': row['file_path']
            }
            
            # Добавляем признаки как отдельные колонки
            for i, feature_value in enumerate(features):
                feature_dict[f'feature_{i}'] = feature_value
            
            feature_list.append(feature_dict)
    
    df_features = pd.DataFrame(feature_list)
    
    return df_features


def merge_with_gdp_data(
    df_features: pd.DataFrame,
    df_gdp: pd.DataFrame
) -> pd.DataFrame:
    """Объединяет признаки с данными о ВВП.
    
    Args:
        df_features: DataFrame с признаками изображений
        df_gdp: DataFrame с данными о ВВП
        
    Returns:
        Объединенный DataFrame
    """
    
    # Подготавливаем данные ВВП
    df_gdp_filtered = df_gdp[['code', 'year', 'gdp']].copy()
    df_gdp_filtered = df_gdp_filtered.dropna(subset=['gdp'])
    
    # Переименовываем для объединения
    df_gdp_filtered = df_gdp_filtered.rename(columns={'code': 'country_code'})
    
    # Объединяем датафреймы
    df_merged = pd.merge(
        df_features,
        df_gdp_filtered,
        on=['country_code', 'year'],
        how='inner'
    )
    
    print(f"Объединено записей: {len(df_merged)}")
    print(f"Уникальных стран: {df_merged['country_code'].nunique()}")
    print(f"Диапазон лет: {df_merged['year'].min()} - {df_merged['year'].max()}")
    
    return df_merged


def split_data_by_country_and_year(
    df: pd.DataFrame,
    test_size: float = 0.2,
    val_size: float = 0.1
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Разделяет данные на train/val/test с учетом стран и временных периодов.
    
    Args:
        df: DataFrame с данными
        test_size: Доля тестовой выборки
        val_size: Доля валидационной выборки
        
    Returns:
        Кортеж (train_df, val_df, test_df)
    """
    
    # Создаем группы по странам
    groups = df['country_code'].values
    
    # Первое разделение: выделяем тестовую выборку
    gss1 = GroupShuffleSplit(
        n_splits=1,
        test_size=test_size,
        random_state=RANDOM_STATE
    )
    
    train_val_idx, test_idx = next(
        gss1.split(df, groups=groups)
    )
    
    train_val_df = df.iloc[train_val_idx]
    test_df = df.iloc[test_idx]
    
    # Второе разделение: выделяем валидационную из train
    val_relative_size = val_size / (1 - test_size)
    gss2 = GroupShuffleSplit(
        n_splits=1,
        test_size=val_relative_size,
        random_state=RANDOM_STATE
    )
    
    train_idx, val_idx = next(
        gss2.split(
            train_val_df,
            groups=train_val_df['country_code'].values
        )
    )
    
    train_df = train_val_df.iloc[train_idx]
    val_df = train_val_df.iloc[val_idx]
    
    print(f"Train: {len(train_df)} samples, {train_df['country_code'].nunique()} countries")
    print(f"Val: {len(val_df)} samples, {val_df['country_code'].nunique()} countries")
    print(f"Test: {len(test_df)} samples, {test_df['country_code'].nunique()} countries")
    
    return train_df, val_df, test_df


def prepare_X_y(
    df: pd.DataFrame,
    feature_cols: List[str],
    target_col: str = 'gdp'
) -> Tuple[np.ndarray, np.ndarray]:
    """Подготавливает матрицу признаков и целевую переменную.
    
    Args:
        df: DataFrame с данными
        feature_cols: Список колонок с признаками
        target_col: Название колонки с целевой переменной
        
    Returns:
        Кортеж (X, y)
    """
    
    X = df[feature_cols].values
    y = df[target_col].values
    
    return X, y


def mean_absolute_percentage_error(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Вычисляет Mean Absolute Percentage Error (MAPE).
    
    Args:
        y_true: Фактические значения
        y_pred: Предсказанные значения
        
    Returns:
        MAPE в процентах
    """
    # Избегаем деления на ноль
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Фильтруем нулевые значения
    mask = y_true != 0
    if not mask.any():
        return np.inf
    
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    return mape


def train_and_evaluate_model(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    feature_cols: List[str],
    model_type: str = 'linear'
) -> Dict[str, float]:
    """Обучает и оценивает модель (линейная регрессия или Random Forest).
    
    Args:
        train_df: Обучающая выборка
        val_df: Валидационная выборка
        test_df: Тестовая выборка
        feature_cols: Список колонок с признаками
        model_type: Тип модели ('linear' или 'random_forest')
        
    Returns:
        Словарь с метриками
    """
    
    # Подготавливаем данные
    X_train, y_train = prepare_X_y(train_df, feature_cols)
    X_val, y_val = prepare_X_y(val_df, feature_cols)
    X_test, y_test = prepare_X_y(test_df, feature_cols)
    
    # Масштабируем признаки (только для линейной регрессии)
    if model_type == 'linear':
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)
    else:
        # Для Random Forest масштабирование не требуется
        X_train_scaled = X_train
        X_val_scaled = X_val
        X_test_scaled = X_test
        scaler = None
    
    # Выбираем и обучаем модель
    if model_type == 'linear':
        model = LinearRegression()
    elif model_type == 'random_forest':
        model = RandomForestRegressor(
            n_estimators=100,      # Количество деревьев
            max_depth=10,           # Максимальная глубина
            min_samples_split=5,    # Минимальное количество样本 для разделения
            min_samples_leaf=2,     # Минимальное количество样本 в листе
            n_jobs=-1,              # Использовать все ядра процессора
            random_state=42,
            verbose=0               # Отключаем вывод прогресса
        )
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")
    
    # Обучаем модель
    print(f"Обучение модели {model_type}...")
    model.fit(X_train_scaled, y_train)
    
    # Делаем предсказания
    y_train_pred = model.predict(X_train_scaled)
    y_val_pred = model.predict(X_val_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Вычисляем метрики
    metrics = {
        # R2
        'train_r2': r2_score(y_train, y_train_pred),
        'val_r2': r2_score(y_val, y_val_pred),
        'test_r2': r2_score(y_test, y_test_pred),
        
        # RMSE
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_val_pred)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        
        # MAPE
        'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
        'val_mape': mean_absolute_percentage_error(y_val, y_val_pred),
        'test_mape': mean_absolute_percentage_error(y_test, y_test_pred)
    }
    
    # Для Random Forest добавляем важность признаков
    if model_type == 'random_forest':
        # Получаем важность признаков
        feature_importance = pd.DataFrame({
            'feature': feature_cols,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nТоп-10 наиболее важных признаков:")
        print(feature_importance.head(10))
        
        # Добавляем информацию о важности признаков в метрики
        metrics['top_features'] = feature_importance.head(10).to_dict()
    
    return metrics, model, scaler


def plot_predictions(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    title: str = "Предсказанные vs Фактические значения"
) -> None:
    """Строит график предсказанных vs фактических значений.
    
    Args:
        y_true: Фактические значения
        y_pred: Предсказанные значения
        title: Заголовок графика
    """
    
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.5)
    
    # Линия идеального предсказания
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    
    plt.xlabel('Фактические значения')
    plt.ylabel('Предсказанные значения')
    plt.title(title)
    plt.tight_layout()
    plt.show()

#### 5.1. Data preprocessing

In [ ]:
print("=" * 50)
print("ШАГ 1: Создание датафрейма изображений")
print("=" * 50)

df_img = create_image_dataframe(IMG_PATH)
print(f"Найдено изображений: {len(df_img)}")
print(f"Уникальных стран: {df_img['country_code'].nunique()}")
print(f"Диапазон лет: {df_img['year'].min()} - {df_img['year'].max()}")

In [ ]:
df_img.head()

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 2: Извлечение признаков из изображений")
print("=" * 50)

df_features = prepare_features_for_country_year(df_img, method='flatten', limit=LIMIT_IMGS)
print(f"Размерность признаков: {df_features.shape}")

In [ ]:
df_features.head()

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 3: Объединение с данными ВВП")
print("=" * 50)

df_merged = merge_with_gdp_data(df_features, df_gdp)

In [ ]:
df_merged.head()

#### 5.2. Training and evaluating

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 4: Разделение на выборки")
print("=" * 50)

train_df, val_df, test_df = split_data_by_country_and_year(df_merged)

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 5: Обучение модели")
print("=" * 50)

# Определяем колонки с признаками
feature_cols = [col for col in df_merged.columns 
                if col.startswith('feature_')]

metrics, model, scaler = train_and_evaluate_model(
    train_df, val_df, test_df, feature_cols, model_type='linear'
)

print("\nМетрики модели:")
for metric_name, metric_value in metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

print("\n" + "=" * 50)
print("ШАГ 6: Визуализация результатов")
print("=" * 50)

# Предсказания на тестовой выборке
X_test, y_test = prepare_X_y(test_df, feature_cols)
X_test_scaled = scaler.transform(X_test)
y_test_pred = model.predict(X_test_scaled)

plot_predictions(y_test, y_test_pred, "Тестовая выборка")

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 7: Обучение модели Random Forest")
print("=" * 50)

# Определяем колонки с признаками
feature_cols = [col for col in df_merged.columns 
                if col.startswith('feature_')]

metrics, model, scaler = train_and_evaluate_model(
    train_df, val_df, test_df, feature_cols, model_type='random_forest'
)

print("\nМетрики модели:")
for metric_name, metric_value in metrics.items():
    try:
        print(f"{metric_name}: {metric_value:.4f}")
    except: pass

print("\n" + "=" * 50)
print("ШАГ 8: Визуализация результатов")
print("=" * 50)

# Предсказания на тестовой выборке
X_test, y_test = prepare_X_y(test_df, feature_cols)
y_test_pred = model.predict(X_test_scaled)

plot_predictions(y_test, y_test_pred, "Тестовая выборка")